# Libraries

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import data

In [24]:
%run Data_preprocessing.ipynb

0
0
0


In [25]:
X_valid

LEVEL2               alanine aminotransferase                                \
Aggregation Function                     mask                                 
hours_in                                    0    1    2    3    4    5    6   
4_185777_294638                           0.0  0.0  0.0  0.0  0.0  1.0  0.0   
21_109451_217847                          0.0  0.0  0.0  0.0  0.0  0.0  0.0   
25_129635_203487                          0.0  0.0  0.0  0.0  0.0  0.0  0.0   
114_178393_258626                         0.0  0.0  0.0  0.0  0.0  0.0  0.0   
129_164174_238431                         1.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                                       ...  ...  ...  ...  ...  ...  ...   
99823_179720_296047                       1.0  0.0  0.0  0.0  0.0  1.0  1.0   
99868_177777_231346                       1.0  0.0  1.0  1.0  0.0  0.0  0.0   
99899_188409_245756                       1.0  0.0  0.0  0.0  0.0  0.0  0.0   
99901_131711_226541                       0.0  0.0  0.0  0.0  0.0  0.0  0.0   
99936_107913_213906                       0.0  0.0  0.0  0.0  0.0  0.0  0.0   

LEVEL2                               ... white blood cell count urine         \
Aggregation Function                 ...          time_since_measured          
hours_in                7    8    9  ...                           14     15   
4_185777_294638       0.0  0.0  0.0  ...                        100.0  100.0   
21_109451_217847      1.0  0.0  0.0  ...                        100.0  100.0   
25_129635_203487      0.0  0.0  0.0  ...                        100.0  100.0   
114_178393_258626     0.0  0.0  0.0  ...                        100.0    0.0   
129_164174_238431     0.0  0.0  0.0  ...                         23.0   24.0   
...                   ...  ...  ...  ...                          ...    ...   
99823_179720_296047   0.0  0.0  0.0  ...                        152.0  153.0   
99868_177777_231346   1.0  1.0  0.0  ...                        176.0  177.0   
99899_188409_245756   0.0  0.0  0.0  ...                         14.0   15.0   
99901_131711_226541   0.0  0.0  0.0  ...                         38.0   39.0   
99936_107913_213906   0.0  0.0  0.0  ...                         62.0   63.0   

LEVEL2                                                                        
Aggregation Function                                                          
hours_in                 16     17     18     19     20     21     22     23  
4_185777_294638       100.0  100.0  100.0  100.0  100.0  100.0  100.0  100.0  
21_109451_217847      100.0  100.0  100.0  100.0  100.0  100.0  100.0  100.0  
25_129635_203487      100.0  100.0  100.0  100.0  100.0  100.0  100.0  100.0  
114_178393_258626       1.0    2.0    3.0    4.0    5.0    6.0    7.0    8.0  
129_164174_238431      25.0   26.0   27.0   28.0   29.0   30.0   31.0   32.0  
...                     ...    ...    ...    ...    ...    ...    ...    ...  
99823_179720_296047   154.0  155.0  156.0  157.0  158.0  159.0  160.0  161.0  
99868_177777_231346   178.0  179.0  180.0  181.0  182.0  183.0  184.0  185.0  
99899_188409_245756    16.0   17.0   18.0   19.0   20.0   21.0   22.0   23.0  
99901_131711_226541    40.0   41.0   42.0   43.0   44.0   45.0   46.0   47.0  
99936_107913_213906    64.0   65.0   66.0   67.0   68.0   69.0   70.0   71.0  

[2394 rows x 7488 columns]

In [6]:
len(X_valid.columns)

7488

# Hyperparameters

In [7]:
input_size = len(X_valid.columns)/24 # 7488
sequence_length = 24
output_size = 2 # live or dead
num_layers = 2
hidden_size=256
learning_rate = 0.01
batch_size = 64
num_epochs = 2

In [8]:
7488/24

312.0

# Create RNN

In [9]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_epochs)
        # N * time_seq * features
        
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size).to(device)
        out, _ = self.rnn(x,h0)
        out = out.reshape(out.shape[0],-1)
        out = self.fc(out)
        return out

# Load data

In [10]:
class CusDatasetLoader(Dataset):
    def __init__(self,dataset):
        xy = dataset
        #np.concatenate((X_train,y_train_t1),axis=1)
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:,0:-1]).to(torch.float32)
        self.y_data = torch.from_numpy(xy[:,[-1]]).to(torch.float32)
  
    def __len__(self):
        return self.len

    def __getitem__(self, index):
        #return self.dataframe.iloc[index]
        return self.x_data[index], self.y_data[index]

In [40]:
X_train_tensor = torch.from_numpy(np.array(X_train)).to(torch.float32)
print(X_train_tensor.shape)
X_train_tensor = torch.split(X_train_tensor,24,1)
X_train_tensor = torch.stack(X_train_tensor)
X_train_tensor.shape

torch.Size([16760, 7488])


torch.Size([312, 16760, 24])

In [60]:
xy = np.concatenate((X_train,y_train_t1.iloc[:,[-1]]),axis=1)

In [80]:
xy = xy.astype(float)
train_datasets = CusDatasetLoader(xy)
train_loader = DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)

In [62]:
x,y = train_datasets[0]
print(x.shape)
print(y.shape)

torch.Size([7488])
torch.Size([1])


In [63]:
model = RNN(input_size, hidden_size, num_layers, output_size).to(device)

In [64]:
criterion = nn.CrossEntropyLoss
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [83]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.view(len(data), 1, -1)
        print(data.shape)
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()
        
        

torch.Size([64, 1, 7488])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x256 and 6144x2)

#

#

#

#

#

#

#

#